## 🧠 Copilot Prompt

Generate a GitHub Actions workflow that automatically runs this notebook using `jupyter nbconvert --execute` and exports the HTML to `docs/validation.html`.

Ensure that `mpmath`, `sympy`, and `numpy` are installed beforehand.

If execution fails, output a warning with traceback.

In [ ]:
import mpmath as mp
import numpy as np
import time
import os

# Set precision and parameters
mp.mp.dps = 25
P = int(os.getenv('PRIME_COUNT', 100))
N = int(os.getenv('ZERO_COUNT', 100))
T = int(os.getenv('INTEGRATION_T', 10))

# Load precomputed zeros
zeros = []
try:
    with open('zeros/zeros_t1e8.txt', 'r') as f:
        zeros = [mp.mpf(line.strip()) for line in f][:N]
    print(f"Loaded {len(zeros)} zeros from file")
except FileNotFoundError:
    print("Warning: zeros_t1e8.txt not found. Computing subset.")
    zeros = [mp.mpf(14.134725 + 2 * np.pi * n / np.log(n+1)) for n in range(N)]
    print(f"Generated {len(zeros)} approximate zeros")

start_time = time.time()
print(f"Starting validation at {time.ctime(start_time)}")


In [ ]:
def prime_sum(f, P):
    """Calculate sum over primes using test function f"""
    total = mp.mpf(0)
    count = 0
    for p in range(2, P + 1):
        # Simple primality test for small numbers
        is_prime = True
        if p > 1:
            for i in range(2, int(p**0.5) + 1):
                if p % i == 0:
                    is_prime = False
                    break
        if is_prime and p > 1:
            total += mp.log(p) * f(mp.log(p))
            count += 1
    print(f"Processed {count} primes up to {P}")
    return total

def zero_sum(f, zeros):
    """Calculate sum over zeros using test function f"""
    total = mp.mpf(0)
    for z in zeros:
        total += f(z)
    print(f"Processed {len(zeros)} zeros")
    return total

# Define test function
test_function = lambda x: mp.exp(-x**2)

# Calculate prime and zero sums
print("\nCalculating prime sum...")
ps = prime_sum(test_function, P)

print("\nCalculating zero sum...")
zs = zero_sum(test_function, zeros)

# Display results
print(f"\n=== VALIDATION RESULTS ===")
print(f"Prime sum: {float(ps):.6f}")
print(f"Zero sum:  {float(zs):.6f}")
print(f"Difference: {float(abs(ps - zs)):.2e}")
print(f"Relative error: {float(abs(ps - zs) / abs(ps)):.2e}" if abs(ps) > 0 else "Relative error: inf")

end_time = time.time()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")
print(f"Parameters used: P={P}, N={N}, T={T}")
